# AAM Demo Script


## Initialization and loading data

In [ ]:
import numpy as np
import scipy.io as sio
import scipy
import matplotlib.pyplot as plt
import math

IMG_SET_ID = 4
path = "data/"
dist_name = path + "distances_" + str(IMG_SET_ID) + ".mat"
PSF_name  = path + "GaussStd2Color_" + str(IMG_SET_ID) + ".mat"
PSF_NIR_name = path + "GaussStd2Nir_" + str(IMG_SET_ID) + ".mat"

mat_dist = sio.loadmat(dist_name)
mat_PSF = sio.loadmat(PSF_name)
mat_PSF_NIR = sio.loadmat(PSF_NIR_name)

distances = mat_dist['distancesCol']
PSF       = mat_PSF['GaussStd2Color']
PSF_NIR   = np.squeeze(mat_PSF_NIR['GaussStd2Nir'])
PSF[:,3] = PSF_NIR

assert distances.shape[0] == PSF.shape[0]
assert distances.shape[0] == PSF_NIR.shape[0]
print('loaded experimental data from {} distances and {} channels.'.format(*PSF.shape))

#distances2 = np.squeeze(distances)
#diff_dist = [x - distances2[i - 1] for i, x in enumerate(distances2) if i > 0]
#b = np.zeros((50, 2))
#b[:,0] = diff_dist
#b[:,1] = range(len(diff_dist))
#print(b)

## Resample data uniformly. 

In [ ]:
from scipy.interpolate import interp1d

num_samples = 50
num_channels = PSF.shape[1]

distances_uniform = np.linspace(distances[0], distances[-1], num=num_samples, endpoint=True)
PSF_uniform = np.empty((num_samples, num_channels))    
for i in range(num_channels):
    x = np.squeeze(distances)
    y = PSF[:,i]
    f = interp1d(x, y, kind='cubic')
    PSF_uniform[:, i] = f(distances_uniform)
    
plt.plot(distances, PSF, '*') 
plt.ylabel('Nonuniform Experimental PSF')
plt.show()

plt.figure()
plt.plot(distances_uniform, PSF_uniform, '*')
plt.ylabel('Interpolated PSF')
plt.show()

In [ ]:
if IMG_SET_ID == 4:
    INDICES_4 = np.array([-1, 0, 1, 2, 3, 4, 5, 6, 7, 9, 13, 17, 19, 20, 21, 22, 23, 24, 25, 27, 31, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])
    INDICES_4 += 1
    PSF_uniform_old = np.zeros((len(INDICES_4),4))
    distances_uniform_old = np.zeros((len(INDICES_4),1))
    idx_uniform = 0
    for idx in range(len(INDICES_4)):
        PSF_uniform_old[idx,:] = PSF[ INDICES_4[idx], : ]
        distances_uniform_old[idx] = distances[ INDICES_4[idx] ]

if IMG_SET_ID == 6 or IMG_SET_ID == 7:
    PSF_uniform_old = PSF
    distances_uniform_old = distances

plt.figure()
plt.plot(distances_uniform_old, PSF_uniform_old, '*')
plt.ylabel('Manual resampled PSF')
plt.show()

In [ ]:
# Polynomial fitting each curve
x = np.squeeze(0.001*distances_uniform)
degree = 4
num_colors = 3
#polyParams = np.zeros((N,deg+1))

polyParams = np.polyfit(x, PSF_uniform, degree).T
print('Fitted {} channels with {}-1 degree polynoms.'.format(*polyParams.shape))

In [ ]:
# Finding x0, focal distance of each color channel.
x0 = np.zeros(num_colors)
for channelIdx in range(num_colors):
    f = np.poly1d( polyParams[channelIdx, :] )
    result = scipy.optimize.minimize_scalar(f, bounds=(x[0], x[-1]), method='bounded')
    x0[channelIdx] = result.x

print(x0)

In [ ]:
# Compute c_ij & d_ij
num_pairs = int(0.5*num_colors*(num_colors-1))
c_ij = np.zeros((num_pairs, degree+1))
d = np.zeros((num_pairs, 2*degree+1))
duo = 0 #corresponds to ij pairs in vectorized form
for i in range(num_colors):
    for j in range(i+1,num_colors):
        polydiff = polyParams[i, :] - polyParams[j, :]
        dtest = np.polymul(polydiff, polydiff)
        #c_ij[duo, :] = polyParams[i, :] - polyParams[j, :]
        #for k in range(2*degree+1):
        #    d[duo, k] = 0
        #    for u in range(degree+1):
        #        for v in range(degree+1):
        #            if u+v == k:
        #                d[duo, k] += c_ij[duo, u] * c_ij[duo, v]
        #print(d[duo] - dtest)
        d[duo] = dtest
        duo += 1

In [ ]:
# Compute the AAM for varying alpha
alphaLIST = np.linspace(0.2, 0.5, 50)
AAM = np.zeros(len(alphaLIST))

# B_ij  RG   RB   Rnum_colors   GB   Gnum_colors   Bnum_colors
# WHY? 
bandwidths = np.array([[400, 500],[500,600],[550,650]])


B_ij = [0.2, 0.3, 0.4, 0.2, 0.5, 0.6]
if num_colors == 3:
    # B_ij  RG   RB   GB
    B_ij = [0.2, 0.3, 0.2]

#def compute_AAM(alpha, )

for alphaIdx, alpha in enumerate(alphaLIST):
    # Computing delta_alpha bar
    duo = 0 #corresponds to ij pairs in vectorized form
    for i in range(num_colors):
        for j in range(i+1,num_colors):
            a_ij = (1 - alpha) * min(x0[i], x0[j])
            b_ij = (1 + alpha) * max(x0[i], x0[j])

            B_ij = max(bandwidths[i,1], bandwidths[j,1]) - min(bandwidths[i,0], bandwidths[j,0])
            delta = 0
            for k in range(1, 2*degree+1+1):
                delta += d[duo, k-1] * (b_ij**k - a_ij**k) / k

            AAM[alphaIdx] += B_ij / (b_ij - a_ij) * delta
            duo += 1

AAM /= num_pairs * 100000

plt.plot(alphaLIST, AAM, 'r^', label="y reconst")
plt.ylabel('AAM')
plt.xlabel('alpha')
plt.show()

print([alphaLIST[0], AAM[0]])
print([alphaLIST[25], AAM[25]])
print([alphaLIST[49], AAM[49]])

In [ ]:
plotIdx = 1

f_poly = np.poly1d(polyParams[plotIdx, :])
x = distances_uniform
y_poly = f_poly(x)

y = PSF_uniform[:, plotIdx]

plt.plot(x, y_poly, 'r^', label="y reconst")
plt.plot(x, y, 'g^', label="y")
plt.ylabel('Experimental PSF')
plt.legend()
plt.show()

In [ ]:
from scipy.optimize import curve_fit

def fit_PSF_func(x, L, d, f, c):
    return np.transpose(np.array(  (L * abs( 1 - d/f + d/x ) + c)  ))

x = distances_uniform
y = PSF_uniform[:, plotIdx]

#params = curve_fit(fit_PSF_func, x, y, p0=(140989., 1.055, 1.054, 2.054))
#params = curve_fit(fit_PSF_func, x, y, bounds=((0,0,0,-np.inf),(np.inf, np.inf, np.inf, np.inf)), p0=(140989., 1.055, 1.054, 2.054))
params = curve_fit(fit_PSF_func, x, y, bounds=((0,0,0,-np.inf),(np.inf, np.inf, np.inf, np.inf)), p0=(70989., 1.055, 1.054, 2.054))

simpleLensParams = params[0]
PSF_fitted = fit_PSF_func(x, *simpleLensParams) 
#PARAMS_GREEN_SET4 = [1.30744663e+05   1.12823345e+00   1.12738942e+00  -2.10215811e-01]

err_fit = np.linalg.norm(PSF_fitted - y)
plt.plot(x, PSF_fitted, 'g^', label=str(err_fit))
plt.plot(x, y, 'k^')
plt.ylabel('Experimental PSF')
plt.legend()
plt.show()

In [ ]:
from scipy.optimize import least_squares
from scipy.optimize import leastsq

def fit_PSF_loss(opt_vector, x, y):
    L, d, f = opt_vector
    #return L * abs( 1 - d/f + d/x )
    y_fit = (L * abs( 1 - d/f + d/x ))
    diff  = y_fit.reshape((-1,)) - y
    return diff

channelIdx = 3

x = distances_uniform
y = PSF_uniform[:, channelIdx]
x0 = np.array([1, 2, 3])
params = leastsq(fit_PSF_loss, x0=x0, args=(x,y), xtol=1e-15)
simpleLensParams = params[0]

PSF_fitted = fit_PSF_loss(simpleLensParams, x, y)

plt.plot(x, PSF_fitted, 'g^')
plt.ylabel('Experimental PSF')
plt.show()
